<a href="https://colab.research.google.com/github/Junhojuno/DeepLearning-Beginning/blob/master/code_review/1.MLP_mnist_tensorflow_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [10]:
# parameter setting
n_input = 784 # 28 x 28 image,  input layer nodes
n_hidden1 = 256 # the number of 1st hidden layer nodes
n_hidden2 = 128 # the number of 2nd hidden layer nodes
n_classes = 10 # 0~9 classification , output layer nodes

# placeholder x, y that we input
x = tf.placeholder(dtype=tf.float32, shape=[None, n_input]) # ?(batch size) x 784
y = tf.placeholder(dtype=tf.float32, shape=[None, n_classes]) # ?(batch size) x 10

# network parameter setting : weights & biases
# weights는 inner product관점에서 생각해보면 이해할 수 있음.
# biases는 각 layer의 node갯수
# input : ? x 784
weights = {
    'h1' : tf.Variable(initial_value=tf.random_normal(shape=[n_input, n_hidden1], stddev=0.1)), # 784 x 256
    'h2' : tf.Variable(initial_value=tf.random_normal(shape=[n_hidden1, n_hidden2], stddev=0.1)), # 256 x 128
    'out' : tf.Variable(initial_value=tf.random_normal(shape=[n_hidden2, n_classes], stddev=0.1)) # 128 x 10
}

biases = {
    'h1' : tf.Variable(initial_value=tf.random_normal(shape=[n_hidden1])), # 256
    'h2' : tf.Variable(initial_value=tf.random_normal(shape=[n_hidden2])), # 128
    'out' : tf.Variable(initial_value=tf.random_normal(shape=[n_classes])) # 10
}

print("parameter setting is completed")

parameter setting is completed


In [13]:
# define graph
# 연산과 연산의 연결을 만든다.

# model
# 위에서 정의한 변수들을 고려하여, 실제 연산 network를 정의한다.
# matrix product, add biases, activation function을 고려한다.
def mlp(_x, _w, _b):
    # tf.sigmoid = tf.nn.sigmoid : 둘의 차이는 없다고 한다.
    hidden_layer1 = tf.nn.sigmoid(tf.add(tf.matmul(_x, _w['h1']),_b['h1']))
    hidden_layer2 = tf.nn.sigmoid(tf.add(tf.matmul(hidden_layer1, _w['h2']),_b['h2']))
    out = tf.add(tf.matmul(hidden_layer2, _w['out']), _b['out'])
    return out # logit으로 출력

# prediction
prediction = mlp(x, weights, biases)

# cost(loss) function, optimizer
# loss : cross entropy
# reduce_sum 까지는 데이터 하나당 loss
# reduce_mean까지 보면 전체 데이터의 loss 평균
# cost = -tf.reduce_mean(tf.reduce_sum(y * tf.log(prediction),axis=1))
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y, logits=prediction))

optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

# accuracy check
# tf.argmax는 가장큰 값의 인덱스 추출
# 이게 맞으면 1, 틀리면 0 추출(tf.equal)
# float로 type변경하여 정확도 출력(boolean type --> float type)
correct = tf.equal(tf.argmax(prediction,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct,dtype='float'))

print("Graph ready..")

Graph ready..


In [14]:
# Training
training_epochs = 20
batch_size = 100
display_step = 4

# initialize
sess = tf.Session()
sess.run(tf.global_variables_initializer())

# optimizing
for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(mnist.train.num_examples / batch_size)
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size=batch_size)
        feed_dict = {x:batch_xs, y:batch_ys}
        sess.run(optimizer, feed_dict=feed_dict)
        avg_cost += sess.run(cost, feed_dict=feed_dict)
    avg_cost = avg_cost / total_batch # epoch당 cost 평균
    # Display
    if (epoch+1) % display_step == 0:
        print("Epoch: %03d/%03d cost: %.9f" % (epoch, training_epochs, avg_cost))
        feed_dict = {x: batch_xs, y: batch_ys}
        train_acc = sess.run(accuracy, feed_dict=feed_dict)
        print("Train Accuracy: %.3f" % (train_acc))
        feed_dict = {x: mnist.test.images, y: mnist.test.labels}
        test_acc = sess.run(accuracy, feed_dict=feed_dict)
        print("Test Accuracy: %.3f" % (test_acc))
        
print("Optimizing Finished!!")

Epoch: 003/020 cost: 0.343062357
Train Accuracy: 0.860
Test Accuracy: 0.907
Epoch: 007/020 cost: 0.257410746
Train Accuracy: 0.900
Test Accuracy: 0.927
Epoch: 011/020 cost: 0.213460426
Train Accuracy: 0.970
Test Accuracy: 0.936
Epoch: 015/020 cost: 0.180081913
Train Accuracy: 0.950
Test Accuracy: 0.944
Epoch: 019/020 cost: 0.153539784
Train Accuracy: 0.950
Test Accuracy: 0.950
Optimizing Finished!!
